In [14]:
# script that allows for sentence transformers based search

from sentence_transformers import SentenceTransformer
from datasets import load_dataset

model = SentenceTransformer('msmarco-MiniLM-L6-cos-v5')
# https://huggingface.co/sentence-transformers/msmarco-MiniLM-L-6-v3

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/627 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/430 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
df = load_dataset("arjunpatel/best-selling-video-games")
df.set_format("pandas")
df = df["train"][:]


In [15]:
#Our sentences we like to encode

#Sentences are encoded by calling model.encode()
sentence_embeddings = model.encode(df.wiki_page, convert_to_tensor=True)

Using custom data configuration arjunpatel--best-selling-video-games-69432951af479ee1
Found cached dataset csv (/Users/ArjunPatel/.cache/huggingface/datasets/arjunpatel___csv/arjunpatel--best-selling-video-games-69432951af479ee1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# used ideas from Nima's implementation here: https://huggingface.co/spaces/NimaBoscarino/playlist-generator/blob/main/app.py


# split up wiki into 512 chunks
# embed each chunk
# match with game
# then dedup hits 


In [18]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
from sentence_transformers import util
import torch
queries = ["Find me a mario game with a sports theme", "find me a first person shooter game"]
top_k = min(5, len(sentence_embeddings))
for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, sentence_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(df.wiki_page.tolist()[idx][0:100], "(Score: {:.4f})".format(score))

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """





Query: Find me a mario game with a sports theme

Top 5 most similar sentences in corpus:
Mario Golf: Super Rush is a golf video game developed by Camelot Software Planning and published by  (Score: 0.5693)
Mario Party Superstars is a 2021 party video game developed by NDcube and published by Nintendo for  (Score: 0.5346)
Super Mario Maker 2 is a 2019 side-scrolling platform game and level creation system developed and p (Score: 0.5122)
Mario Strikers: Battle League is a 2022 sports video game developed by Next Level Games and publishe (Score: 0.4952)
Mario + Rabbids Kingdom Battle is a turn-based tactics action-adventure video game developed by Ubis (Score: 0.4818)




Query: find me a first person shooter game

Top 5 most similar sentences in corpus:
Monster Hunter (モンスターハンター, Monsutā Hantā) is a Japanese media franchise centered around a series of  (Score: 0.3304)
Pokémon Mystery Dungeon is a video game series spin-off from the main Pokémon series developed by Sp (Score: 0.3258)
